# Preprocemento - Dataset 1

## Imports

In [1]:
import os
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib as mpl
from sklearn import cluster, datasets, mixture
import matplotlib.pyplot as plt
import random

## Setting random seeds

Força inicialização das tabelas pseudo aleatórias a fim de se ter repetibilidade nos tests

In [3]:
np.random.seed(1)
random.seed(1)

## Leitura dos dados e armazenamento em DataFrame Pandas

In [3]:
dataPath = r'D:\14--OneDrive\OneDrive\ML-SEA-2021\Rede_Teste_2'
npzFname = 'dataset_v1_lhs1000_sw_ar.npz'

npz = np.load(os.path.join(dataPath, npzFname))

In [5]:
npz['columns']

array(['sw', 'ar', 'sweep', 'static_thrust', 'climb_cas', 'climb_mach',
       'cruise_mach', 'cruise_alt', 'mission_dist', 'tow', 'bow',
       'w_fuel', 'd_mission', 't_mission', 'actual_cruise_alt'],
      dtype='<U17')

### Conversão para DataFrame e overview do resultado

In [6]:
df = pd.DataFrame(npz['data'], columns=npz['columns'])
df

,sw,ar,sweep,static_thrust,climb_cas,climb_mach,cruise_mach,cruise_alt,mission_dist,tow,bow,w_fuel,d_mission,t_mission,actual_cruise_alt
0,186.706425,7.018758,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,102802.514008,51804.883466,27016.710450,2000.564005,266.091132,30000.0
1,179.168747,13.208006,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,96416.320220,52941.241191,19508.492083,2002.965022,265.912557,30000.0
2,124.388366,12.311823,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,90075.218293,51300.794817,14792.458241,2002.378584,265.637597,30000.0
3,103.742971,12.325404,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,88937.950310,50754.583475,14202.296172,2002.397955,265.640858,30000.0
4,104.757170,9.920992,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,90034.909365,50488.509865,15583.172871,2002.876260,265.810799,30000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,102.642208,12.284328,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,88910.421265,50720.451519,14209.327810,2002.410929,265.645836,30000.0
996,125.787540,13.909113,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,89904.725258,51571.645887,14346.139136,2002.236822,265.586011,30000.0
997,99.942461,11.034167,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,89305.331055,50503.553604,14829.827686,2002.655070,265.735071,30000.0
998,152.282747,9.547638,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,93724.899818,51551.343512,18217.875996,2003.084133,265.900891,30000.0


## Operações gerais

In [11]:
df.loc[:,'sw']>120

0       True
1       True
2       True
3      False
4      False
       ...  
995    False
996     True
997    False
998     True
999     True
Name: sw, Length: 1000, dtype: bool

In [7]:
df2 = df.loc[df.loc[:,'sw']>120]
df2

,sw,ar,sweep,static_thrust,climb_cas,climb_mach,cruise_mach,cruise_alt,mission_dist,tow,bow,w_fuel,d_mission,t_mission,actual_cruise_alt
0,186.706425,7.018758,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,102802.514008,51804.883466,27016.710450,2000.564005,266.091132,30000.0
1,179.168747,13.208006,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,96416.320220,52941.241191,19508.492083,2002.965022,265.912557,30000.0
2,124.388366,12.311823,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,90075.218293,51300.794817,14792.458241,2002.378584,265.637597,30000.0
6,132.462274,9.493747,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,91785.967731,51080.811873,16743.506601,2002.907898,265.826738,30000.0
7,151.622510,11.015731,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,92971.887124,51794.884494,17207.924130,2002.768495,265.796673,30000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,190.106465,11.474841,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,99054.602856,52859.609934,22206.507455,2000.720524,265.770331,30000.0
992,177.109558,7.379412,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,99918.801907,51683.613123,24272.847064,2001.314600,266.024643,30000.0
996,125.787540,13.909113,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,89904.725258,51571.645887,14346.139136,2002.236822,265.586011,30000.0
998,152.282747,9.547638,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,93724.899818,51551.343512,18217.875996,2003.084133,265.900891,30000.0


In [12]:
df2 = df.loc[(df.loc[:,'sw']>120)|(df.loc[:,'ar']>9)]
df2

,sw,ar,sweep,static_thrust,climb_cas,climb_mach,cruise_mach,cruise_alt,mission_dist,tow,bow,w_fuel,d_mission,t_mission,actual_cruise_alt
0,186.706425,7.018758,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,102802.514008,51804.883466,27016.710450,2000.564005,266.091132,30000.0
1,179.168747,13.208006,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,96416.320220,52941.241191,19508.492083,2002.965022,265.912557,30000.0
2,124.388366,12.311823,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,90075.218293,51300.794817,14792.458241,2002.378584,265.637597,30000.0
3,103.742971,12.325404,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,88937.950310,50754.583475,14202.296172,2002.397955,265.640858,30000.0
4,104.757170,9.920992,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,90034.909365,50488.509865,15583.172871,2002.876260,265.810799,30000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,102.642208,12.284328,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,88910.421265,50720.451519,14209.327810,2002.410929,265.645836,30000.0
996,125.787540,13.909113,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,89904.725258,51571.645887,14346.139136,2002.236822,265.586011,30000.0
997,99.942461,11.034167,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,89305.331055,50503.553604,14829.827686,2002.655070,265.735071,30000.0
998,152.282747,9.547638,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,93724.899818,51551.343512,18217.875996,2003.084133,265.900891,30000.0


In [13]:
df2 = df.loc[(df.loc[:,'sw']>120)|(df.loc[:,'ar']>9)]
df2

,sw,ar,sweep,static_thrust,climb_cas,climb_mach,cruise_mach,cruise_alt,mission_dist,tow,bow,w_fuel,d_mission,t_mission,actual_cruise_alt
0,186.706425,7.018758,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,102802.514008,51804.883466,27016.710450,2000.564005,266.091132,30000.0
1,179.168747,13.208006,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,96416.320220,52941.241191,19508.492083,2002.965022,265.912557,30000.0
2,124.388366,12.311823,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,90075.218293,51300.794817,14792.458241,2002.378584,265.637597,30000.0
3,103.742971,12.325404,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,88937.950310,50754.583475,14202.296172,2002.397955,265.640858,30000.0
4,104.757170,9.920992,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,90034.909365,50488.509865,15583.172871,2002.876260,265.810799,30000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,102.642208,12.284328,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,88910.421265,50720.451519,14209.327810,2002.410929,265.645836,30000.0
996,125.787540,13.909113,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,89904.725258,51571.645887,14346.139136,2002.236822,265.586011,30000.0
997,99.942461,11.034167,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,89305.331055,50503.553604,14829.827686,2002.655070,265.735071,30000.0
998,152.282747,9.547638,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,93724.899818,51551.343512,18217.875996,2003.084133,265.900891,30000.0


In [13]:
array = df2.values
array

array([[1.86706425e+02, 7.01875830e+00, 2.50000000e+01, ...,
        2.00056401e+03, 2.66091132e+02, 3.00000000e+04],
       [1.79168747e+02, 1.32080056e+01, 2.50000000e+01, ...,
        2.00296502e+03, 2.65912557e+02, 3.00000000e+04],
       [1.24388366e+02, 1.23118228e+01, 2.50000000e+01, ...,
        2.00237858e+03, 2.65637597e+02, 3.00000000e+04],
       ...,
       [9.99424610e+01, 1.10341671e+01, 2.50000000e+01, ...,
        2.00265507e+03, 2.65735071e+02, 3.00000000e+04],
       [1.52282747e+02, 9.54763773e+00, 2.50000000e+01, ...,
        2.00308413e+03, 2.65900891e+02, 3.00000000e+04],
       [1.41591488e+02, 1.19580377e+01, 2.50000000e+01, ...,
        2.00252556e+03, 2.65702800e+02, 3.00000000e+04]])

In [20]:
array2 =df[['sw','ar']].values
array2

array([[186.70642506,   7.0187583 ],
       [179.16874671,  13.20800563],
       [124.38836606,  12.31182283],
       ...,
       [ 99.94246103,  11.03416713],
       [152.28274652,   9.54763773],
       [141.59148768,  11.95803773]])

In [22]:
df2 = df.dropna(how='any', axis='index',  inplace=False)
df2

,sw,ar,sweep,static_thrust,climb_cas,climb_mach,cruise_mach,cruise_alt,mission_dist,tow,bow,w_fuel,d_mission,t_mission,actual_cruise_alt
0,186.706425,7.018758,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,102802.514008,51804.883466,27016.710450,2000.564005,266.091132,30000.0
1,179.168747,13.208006,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,96416.320220,52941.241191,19508.492083,2002.965022,265.912557,30000.0
2,124.388366,12.311823,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,90075.218293,51300.794817,14792.458241,2002.378584,265.637597,30000.0
3,103.742971,12.325404,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,88937.950310,50754.583475,14202.296172,2002.397955,265.640858,30000.0
4,104.757170,9.920992,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,90034.909365,50488.509865,15583.172871,2002.876260,265.810799,30000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,102.642208,12.284328,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,88910.421265,50720.451519,14209.327810,2002.410929,265.645836,30000.0
996,125.787540,13.909113,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,89904.725258,51571.645887,14346.139136,2002.236822,265.586011,30000.0
997,99.942461,11.034167,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,89305.331055,50503.553604,14829.827686,2002.655070,265.735071,30000.0
998,152.282747,9.547638,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,93724.899818,51551.343512,18217.875996,2003.084133,265.900891,30000.0


In [14]:
df['newPar'] = 1 /df['ar']
df

,sw,ar,sweep,static_thrust,climb_cas,climb_mach,cruise_mach,cruise_alt,mission_dist,tow,bow,w_fuel,d_mission,t_mission,actual_cruise_alt,newPar
0,186.706425,7.018758,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,102802.514008,51804.883466,27016.710450,2000.564005,266.091132,30000.0,0.142475
1,179.168747,13.208006,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,96416.320220,52941.241191,19508.492083,2002.965022,265.912557,30000.0,0.075712
2,124.388366,12.311823,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,90075.218293,51300.794817,14792.458241,2002.378584,265.637597,30000.0,0.081223
3,103.742971,12.325404,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,88937.950310,50754.583475,14202.296172,2002.397955,265.640858,30000.0,0.081133
4,104.757170,9.920992,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,90034.909365,50488.509865,15583.172871,2002.876260,265.810799,30000.0,0.100796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,102.642208,12.284328,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,88910.421265,50720.451519,14209.327810,2002.410929,265.645836,30000.0,0.081405
996,125.787540,13.909113,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,89904.725258,51571.645887,14346.139136,2002.236822,265.586011,30000.0,0.071895
997,99.942461,11.034167,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,89305.331055,50503.553604,14829.827686,2002.655070,265.735071,30000.0,0.090628
998,152.282747,9.547638,25.0,25000.0,250.0,0.6,0.8,30000.0,2000.0,93724.899818,51551.343512,18217.875996,2003.084133,265.900891,30000.0,0.104738


In [24]:
df.columns

Index(['sw', 'ar', 'sweep', 'static_thrust', 'climb_cas', 'climb_mach',
       'cruise_mach', 'cruise_alt', 'mission_dist', 'tow', 'bow', 'w_fuel',
       'd_mission', 't_mission', 'actual_cruise_alt', 'newPar'],
      dtype='object')

## Mapping functions

https://stackoverflow.com/questions/19798153/difference-between-map-applymap-and-apply-methods-in-pandas

In [29]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'), index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,-0.322417,-0.384054,1.133769
Ohio,-1.099891,-0.172428,-0.877858
Texas,0.042214,0.582815,-1.100619
Oregon,1.144724,0.901591,0.502494


Applying a function on 1D arrays to each column or row

In [31]:
f = lambda x: x.max() - x.min()
frame2 = frame.apply(f)
frame2

b    2.244615
d    1.285645
e    2.234389
dtype: float64

Apply function to each element independtly

In [34]:
fformat = lambda x: '%.2f' % x
frame3 = frame.applymap(fformat)
frame3

,b,d,e
Utah,-0.32,-0.38,1.13
Ohio,-1.10,-0.17,-0.88
Texas,0.04,0.58,-1.10
Oregon,1.14,0.90,0.50


In [35]:
frame['e'].map(format)

Utah       1.1337694423354374
Ohio      -0.8778584179213718
Texas     -1.1006191772129212
Oregon     0.5024943389018682
Name: e, dtype: object